Write an algorithm that extracts / identifies the epithelium layer in the conjunctival lesion image.

In [2]:
import cv2
import numpy as np
from PIL import Image, ImageFile

In [12]:
# WE CAME TO THE CONCLUSION THAT THIS DOESN'T WORK WELL, NOR DOES IT ACHIEVE THE OBJECTIVE RIGHT
  # just wanted to post since some of the processes helped drive some other team members decisionmaking

# upload sample files from auto-script provided by team 2

# image link: https://drive.google.com/drive/folders/1isj_q68hJpH-LOhDiBrZmQL15DZOnAvy?usp=drive_link

# goal this week: out of all the files I now have uploaded, can I match which ones are similar?
  # can we find similarities between other stains as well? Are we able to match similar items regardless of stain to each other?

In [ ]:
# same stain, different slices

image_paths = ['h2114153h&e.tif', 'h2114154h&e.tif', 'h2114154h&e2.tif', 'h2114156h&e.tif', 'h2114156h&e2.tif', 'h2114157h&e.tif', 'h2114158h&e.tif', 'h2114158h&e2.tif']

ImageFile.LOAD_TRUNCATED_IMAGES = True # used to remove errors

def compare_images_pairwise(image_paths, similarity_threshold=0.15):
    orb = cv2.ORB_create()

    descriptors = []
    keypoints = []
    images = []

    # Getting keypoints - this is what I got from what little from my 362 lecture I was referencing today
    for path in image_paths:
        try:
            pil_image = Image.open(path)
            img_rgb = np.array(pil_image.convert('RGB'))

            # Convert the image from RGB to HSV using OpenCV
            img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV) # HSV works a lot better than when I tested RGB
            kp, des = orb.detectAndCompute(img_hsv, None)

            keypoints.append(kp)
            descriptors.append(des)
            images.append(path)
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            return

    # NORM_HAMMING - uses 'distance' as a metric to help reference points
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)

    for i in range(len(descriptors)):
        for j in range(i + 1, len(descriptors)):  # Pairwise comparison
            try:
                # using 2 for now, might need to adjust after further conversations
                matches = bf.knnMatch(descriptors[i], descriptors[j], k=2)

                # ratios are very arbitrary
                good_matches = []
                for m, n in matches:
                    if m.distance < 0.98 * n.distance: # tests a few values here
                        good_matches.append(m)

                # Measure the percentage of good matches relative to keypoints in the first image
                match_ratio = len(good_matches) / len(keypoints[i])
                  # This threshold doesn't mean anything, and we realized it isn't a good determinant

                if match_ratio > similarity_threshold:
                    print(f"Images {images[i]} and {images[j]} are similar (good match ratio: {match_ratio:.2f})")
                else:
                    print(f"Images {images[i]} and {images[j]} are different (good match ratio: {match_ratio:.2f})")
            except Exception as e:
                print(f"Error comparing {images[i]} and {images[j]}: {e}")

compare_images_pairwise(image_paths, similarity_threshold=0.1)


Images h2114153h&e.tif and h2114154h&e.tif are similar (good match ratio: 0.75)
Images h2114153h&e.tif and h2114154h&e2.tif are similar (good match ratio: 0.71)
Images h2114153h&e.tif and h2114156h&e.tif are similar (good match ratio: 0.76)
Images h2114153h&e.tif and h2114156h&e2.tif are similar (good match ratio: 0.73)
Images h2114153h&e.tif and h2114157h&e.tif are similar (good match ratio: 0.75)
Images h2114153h&e.tif and h2114158h&e.tif are similar (good match ratio: 0.74)
Images h2114153h&e.tif and h2114158h&e2.tif are similar (good match ratio: 0.71)
Images h2114154h&e.tif and h2114154h&e2.tif are similar (good match ratio: 0.70)
Images h2114154h&e.tif and h2114156h&e.tif are similar (good match ratio: 0.71)
Images h2114154h&e.tif and h2114156h&e2.tif are similar (good match ratio: 0.74)
Images h2114154h&e.tif and h2114157h&e.tif are similar (good match ratio: 0.73)
Images h2114154h&e.tif and h2114158h&e.tif are similar (good match ratio: 0.76)
Images h2114154h&e.tif and h2114158

In [11]:
# different stains and trying to match to the same slices
# taking everything as grayscale so that just shape and edges are considered, not color gradient or anything like that

image_paths = ['h2114153h&e.tif', 'h2114153melan.tif', 'h2114153melan2.tif', 'h2114153sox10.tif',
               'h2114154h&e.tif', 'h2114154h&e2.tif', 'h2114154melan.tif', 'h2114154melan2.tif', 'h2114154sox10.tif',
               'h2114156h&e.tif', 'h2114156h&e2.tif', 'h2114156melan.tif', 'h2114156melan2.tif', 'h2114156sox10.tif',
               'h2114157h&e.tif', 'h2114157sox10.tif',
               'h2114158h&e.tif', 'h2114158h&e2.tif', 'h2114158melan.tif', 'h2114158melan2.tif']


def compare_images_different_stains(image_paths, similarity_threshold=0.15):
    orb = cv2.ORB_create()

    descriptors = []
    keypoints = []
    images = []

    # Detect and compute keypoints and descriptors for all images
    for path in image_paths:
        try:
            pil_image = Image.open(path)
            img_rgb = np.array(pil_image.convert('RGB'))

            # Reverting back to a more basic structure that we did the first week
            img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

            # Apply Canny edge detection on the grayscale image - like the first week
            img_edges = cv2.Canny(img_gray, 100, 200)

            # Compute keypoints and descriptors using ORB on the edge-detected image
            kp, des = orb.detectAndCompute(img_edges, None)

            keypoints.append(kp)
            descriptors.append(des)
            images.append(path)
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            return

    bf = cv2.BFMatcher(cv2.NORM_HAMMING)

    for i in range(len(descriptors)):
        for j in range(i + 1, len(descriptors)):
            try:
                matches = bf.knnMatch(descriptors[i], descriptors[j], k=2)

                good_matches = []
                for m, n in matches:
                    if m.distance < 0.75 * n.distance:
                        good_matches.append(m)
                match_ratio = len(good_matches) / len(keypoints[i])

                # We realized that this really doesn't help with deciphering as much as we thought
                if match_ratio > similarity_threshold and match_ratio > 0.60:
                    print(f"Images {images[i]} and {images[j]} good match ratio: {match_ratio:.2f}")
            except Exception as e:
                print(f"Error comparing {images[i]} and {images[j]}: {e}")

# Compare images pairwise
compare_images_different_stains(image_paths, similarity_threshold=0.1)
